## Algorithm
The algorithm for solving non-sequential games with pure strategy is simple. It consists of only two functions: "s_matrix" and "best_responses". 

In [7]:
using SymPy #package to use symbolic variables

function s_matrix(x) #function to create the strategy matrix
    S = vec(collect(Base.product(reverse(x)...))) #Cartesian Product
    #transform a set of strategies into a matrix
    S_M = []
    for i = 1:length(S[1])
        d = [y[i] for y in S]
        if i == 1 S_M = [d; ] else S_M = [S_M d] end
    end
    S_M = reverse(S_M, dims = 2)
    return S_M
end

function best_responses(x,y) #S e U
ncol = size(x, 2) #number of players
BR = copy(x) #matrix containing the best responses 
for i = 1:ncol #for each payer i
    S_not_i = sum(x[:, 1:end .!= i], dims=2) # possible strategies of opponents of player i 
    for j in unique(S_not_i) 
        k = findall(S_not_i .== j) #subset of strategies containing strategy j
        l = first.(Tuple.(k)) # lines referring to strategy j 
        h = findall(y[l,i] .== max(maximum(y[l,i]))) #player (i) strategies that maximize their payoff versus strategy J 
        #create matrix of best responses
            for ll in l[h] BR[ll,i] = Sym(string(BR[ll,i],"^*")) end #mark best responses with (*) 
    end
end
return BR
end

best_responses (generic function with 1 method)

## Example: Prisoner's dilemma

Mas-Colell (pg: 236)

In [8]:
#strategy matrix
a1 = [Sym("a_$i^$j") for i=1 for j=["nc","c"]] #possible actions of payer 1
a2 = [Sym("a_$i^$j") for i=2 for j=["nc","c"]] #possible actions of payer 2
A = [a1 , a2] # set of all possible actions
S = s_matrix(A) 

#Payoff matrix
U = [-2 -2; -10 -1;-1 -10;-5 -5]

#Best responses matrix
BR = best_responses(S,U)

4×2 Matrix{Sym}:
  a_1__nc   a_2__nc
  a_1__nc  a_2__c__*
 a_1__c__*   a_2__nc
 a_1__c__*  a_2__c__*

## Example: Strictly dominated Strategy
Mas-Colell (pq: 238)

In [14]:
#strategy matrix
a1 = [Sym("a_$i^$j") for i=1 for j=["U","M","D"]] 
a2 = [Sym("a_$i^$j") for i=2 for j=["L","R"]] 
A = [a1 , a2] 
S = s_matrix(A) 

#Payoff matrix
U = [1 -1;-1 1;-1 1;1 -1;-2 5;-3 2]

#Best responses matrix
BR = best_responses(S,U)

6×2 Matrix{Sym}:
 a_1__U__*    a_2__L
   a_1__U  a_2__R__*
   a_1__M  a_2__L__*
 a_1__M__*    a_2__R
   a_1__D  a_2__L__*
   a_1__D    a_2__R

The player (1) will never take action (D).

## Example: Three Players
Osborne and Rubinstein (1994, pg: 48)

In [17]:
#strategy matrix
a1 = [Sym("a_$i^$j") for i=1 for j=["T","B"]] 
a2 = [Sym("a_$i^$j") for i=2 for j=["L","R"]] 
a3 = [Sym("a_$i^$j") for i=3 for j=["A","B","C"]] 
A = [a1 , a2, a3] 
S = s_matrix(A) 

#Payoff matrix
U = [0 0 3; 2 2 2; 0 0 0; 0 0 0; 0 0 0; 0 0 0; 1 0 0; 0 0 0; 0 1 0; 0 0 0; 2 2 2; 0 0 3]

#Best responses matrix
BR = best_responses(S,U)

12×3 Matrix{Sym}:
   a_1__T  a_2__L__*  a_3__A__*
 a_1__T__*  a_2__L__*    a_3__B
 a_1__T__*  a_2__L__*    a_3__C
 a_1__T__*  a_2__R__*  a_3__A__*
   a_1__T    a_2__R  a_3__B__*
 a_1__T__*  a_2__R__*  a_3__C__*
 a_1__B__*  a_2__L__*  a_3__A__*
   a_1__B    a_2__L  a_3__B__*
 a_1__B__*  a_2__L__*  a_3__C__*
 a_1__B__*  a_2__R__*    a_3__A
 a_1__B__*  a_2__R__*    a_3__B
 a_1__B__*    a_2__R  a_3__C__*

The pure strategy equilibrium strategies are $\left(a_{1}^{T^{*}} , a_{2}^{R^{*}} , a_{3}^{A^{*}}\right)$, $\left(a_{1}^{T^{*}} , a_{2}^{R^{*}} , a_{3}^{C^{*}}\right)$, $\left(a_{1}^{B^{*}} , a_{2}^{L^{*}} , a_{3}^{A^{*}}\right)$, and $\left(a_{1}^{B^{*}} , a_{2}^{L^{*}} , a_{3}^{C^{*}}\right)$. And, the pure strategy equilibrium payoffs are (0,0,0), (1,0,0), and (0,1,0).

## Example: Penalty kicks in soccer (three actions)

https://felixmunozgarcia.files.wordpress.com/2017/08/slides_71.pdf

In [25]:
#strategy matrix
Beth = [Sym("a_$i^$j") for i="B" for j=["f","b"]]
Tommy = [Sym("a_$i^$j") for i="T" for j=["f","b"]]
Jason = [Sym("a_$i^$j") for i="J" for j=["f","b"]]
A = [Beth , Tommy, Jason]
S = s_matrix(A) 

#Payoff matrix
U = [0 0 0; 3 3 -2;-4 1 2;1 -4 2;1 -4 2;-4 1 2; 2 2 -2; 0 0 0]

#Best responses matrix
BR = best_responses(S,U)

8×3 Matrix{Sym}:
   a_B__f    a_T__f  a_J__f__*
 a_B__f__*  a_T__f__*    a_J__b
   a_B__f  a_T__b__*  a_J__f__*
 a_B__f__*    a_T__b  a_J__b__*
 a_B__b__*    a_T__f  a_J__f__*
   a_B__b  a_T__f__*  a_J__b__*
 a_B__b__*  a_T__b__*    a_J__f
   a_B__b    a_T__b  a_J__b__*